### **How trustworthy is your local representative?**

#### Data Sources:
politifact api documentation: http://static.politifact.com/api/v2apidoc.html <br>
govtrack api documentation: https://www.govtrack.us/developers/api <br>
propublica: https://projects.propublica.org/api-docs/congress-api/ <br>
congressional misconduct database: https://www.govtrack.us/misconduct <br>
news api: https://newsapi.org/docs <br>
votesmart: https://github.com/votesmart/python-votesmart/tree/master

In [24]:
import requests
import json
import pandas as pd

In [29]:
# Politifact truth-o-meter data

politifact = 'http://www.politifact.com/api/statements/truth-o-meter/json/?n=100'
resp = requests.get(politifact).json()

sample = pd.DataFrame(resp)

# Quick Cleaning
for i in range(len(sample)):
  
    sample.iloc[i].ruling = sample.iloc[i].ruling['ruling_slug']

    sample.iloc[i].speaker = sample.iloc[i].speaker['name_slug']

    sample.iloc[i].statement_type = sample.iloc[i].statement_type['statement_type']

    sample.iloc[i].statement = (sample.iloc[i].statement
                            .replace('<p>&quot;','')
                            .replace('&quot;</p>\r\n',''))

    subject_list = list()
    
    for subject in sample.iloc[i].subject:
        subject_list.append(subject['subject_slug'])
        sample.iloc[i].subject = subject_list

sample = sample[['ruling','speaker','statement','statement_context','statement_date','statement_type','subject','statement_url']]
sample.head()

,ruling,speaker,statement,statement_context,statement_date,statement_type,subject,statement_url
0,mostly-true,donald-trump,When a car is sent to the United States from C...,in a tweet,2018-04-09,Claim,"[china, trade]",/truth-o-meter/statements/2018/apr/09/donald-t...
1,barely-true,donald-trump,This will be the last time &mdash; April &mdas...,an event in West Virginia,2018-04-05,Claim,[taxes],/truth-o-meter/statements/2018/apr/09/donald-t...
2,pants-fire,blog-posting,YouTube witness makes shocking admission on li...,a fake news blog post,2018-04-04,Claim,[fake-news],/truth-o-meter/statements/2018/apr/06/blog-pos...
3,pants-fire,donald-trump,"In many places, like California, the same pers...",a speech in West Virginia,2018-04-05,Claim,[elections],/truth-o-meter/statements/2018/apr/06/donald-t...
4,false,ron-wyden,Republicans in Congress are plotting to take a...,a tweet,2018-03-29,Claim,"[federal-budget, medicaid, medicare, social-se...",/truth-o-meter/statements/2018/apr/05/ron-wyde...


In [30]:
# Data on legislators

person = 'https://www.govtrack.us/api/v2/person'
role = 'https://www.govtrack.us/api/v2/role'

params = {
    'current' : 'true'
}

resp = requests.get(person).json()

legis = pd.DataFrame(resp['objects'])
legis.head()

,bioguideid,birthday,cspanid,firstname,gender,gender_label,lastname,link,middlename,name,namemod,nickname,osid,pvsid,sortname,twitterid,youtubeid
0,A000069,1924-09-11,8704.0,Daniel,male,Male,Akaka,https://www.govtrack.us/congress/members/danie...,Kahikina,"Sen. Daniel Akaka [D-HI, 1990-2012]",,,N00007653,53286,"Akaka, Daniel (Sen.) [D-HI, 1990-2012]",None,None
1,A000360,1940-07-03,5.0,Lamar,male,Male,Alexander,https://www.govtrack.us/congress/members/lamar...,,Sen. Lamar Alexander [R-TN],,,N00009888,15691,"Alexander, Lamar (Sen.) [R-TN]",SenAlexander,lamaralexander
2,A000109,1943-12-02,NaN,Wayne,male,Male,Allard,https://www.govtrack.us/congress/members/wayne...,A.,"Sen. Wayne Allard [R-CO, 1997-2008]",,,N00009082,26783,"Allard, Wayne (Sen.) [R-CO, 1997-2008]",None,None
3,A000121,1952-03-08,NaN,George,male,Male,Allen,https://www.govtrack.us/congress/members/georg...,Felix,"Sen. George Allen [R-VA, 2001-2006]",,,N00009957,22404,"Allen, George (Sen.) [R-VA, 2001-2006]",None,None
4,B000243,1941-12-11,1727.0,Max,male,Male,Baucus,https://www.govtrack.us/congress/members/max_b...,S.,"Sen. Max Baucus [D-MT, 1978-2014]",,,N00004643,53315,"Baucus, Max (Sen.) [D-MT, 1978-2014]",None,None


In [55]:
from redis import StrictRedis
from congress import Congress
db = StrictRedis()
congress = Congress(propublica_key, cache=db)
senate = congress.members.filter('senate') # hits the API, caching the result
senate = congress.members.filter('senate') # uses the cache

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [50]:
from congress import Congress
propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'
congress = Congress(propublica_key)
congress.members.get('A000069').keys()

dict_keys(['member_id', 'first_name', 'middle_name', 'last_name', 'suffix', 'date_of_birth', 'gender', 'url', 'times_topics_url', 'times_tag', 'govtrack_id', 'cspan_id', 'votesmart_id', 'icpsr_id', 'twitter_account', 'facebook_account', 'youtube_account', 'crp_id', 'google_entity_id', 'rss_url', 'in_office', 'current_party', 'most_recent_vote', 'roles'])

In [40]:
propublica_key = 'wAxQ7sF8gcXCBRnY3lzegT23aljM4saALOb6JPlR'

propublica = 'https://api.propublica.org/congress/v1/'

headers = {"X-API-Key" : propublica_key}

params = {
    'Congress' : 115
}

resp = requests.post(propublica, params=params,headers=headers)

In [41]:
resp.json()

{'message': 'Missing Authentication Token'}